In [5]:
from pathlib import Path
from scanpy import read_h5ad
from umap import UMAP
import pandas as pd
import anndata

h5ad_file_paths = Path('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/cross_areal_old_tax_copies').glob('*/processed/*.h5ad')


In [6]:
 fname_to_section = {'202112101342_H1930001Cx46MTG0202007205_VMSC01001' : 'H19.30.001.Cx46.MTG02.02.007.2.05',
                     '202112021529_H1930001Cx46MTG202007203_VMSC01001' : 'H19.30.001.Cx46.MTG2.02.007.2.03',
                     '202112021532_H1930001Cx46MTG202007204_vmsc00401' : 'H19.30.001.Cx46.MTG2.02.007.2.04',
                     '202201271619_H1930002Cx46MTG202007102_VMSC01001' : 'H19.30.001.Cx46.MTG2.02.007.1.02',
                     '202201281113_H1930002Cx46MTG202007104_vmsc00401' : 'H19.30.001.Cx46.MTG2.02.007.1.04',
                     '202201271624_H1930002Cx46MTG202007105_vmsc00401' : 'H19.30.001.Cx46.MTG2.02.007.1.05',
                     '202207211116_1190312477_VMSC02501' : 'H20.30.002.Cx30.MTG.02.007.1.01.01',
                     '202207221256_1190312488_VMSC01601' : 'H20.30.002.Cx30.MTG.02.007.1.01.03'}

In [7]:
h5ad_raws = []
for file in h5ad_file_paths:
    adata = read_h5ad(file)
    fname = file.stem
    print(fname)
    adata.uns['section'] = fname_to_section[fname]
    adata.obs['section'] = adata.uns['section']
    adata.obs['filename'] = fname
    adata.write_h5ad(file)
    h5ad_raws.append(adata)

combo_anndata = anndata.concat(h5ad_raws, uns_merge='first')
combo_anndata.obsm['X_umap'] = UMAP(n_components=2, n_neighbors = 25, metric='euclidean', min_dist=0.4).fit_transform(combo_anndata.X)

202112021529_H1930001Cx46MTG202007203_VMSC01001
202112021532_H1930001Cx46MTG202007204_vmsc00401
202201271619_H1930002Cx46MTG202007102_VMSC01001
202201281113_H1930002Cx46MTG202007104_vmsc00401
202112101342_H1930001Cx46MTG0202007205_VMSC01001
202201271624_H1930002Cx46MTG202007105_vmsc00401
202207211116_1190312477_VMSC02501
202207221256_1190312488_VMSC01601


In [8]:
#adjust the cirro for the combined file
import numpy as np

counter = 0
for ii, gb in combo_anndata.obs.groupby("section"): #ii = section name, gb = indexed chunk of dataframe
    spatial = combo_anndata.obsm['spatial'][combo_anndata.obs.section==ii,:]
    cirro_y = -(spatial[:,1] - np.mean(spatial[:,1]))
    cirro_x = (spatial[:,0] - min(spatial[:,0])) +(counter*10000)
    counter += 1
    coordinates_cirro = np.column_stack((cirro_x, cirro_y))
    combo_anndata.obsm['spatial_cirro'][np.where(combo_anndata.obs.section==ii)] = coordinates_cirro
combo_anndata.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/cross_areal_old_tax_copies/Cross_areal_old_tax_copies.h5ad')


In [91]:
# #concatenate here
# h5ad_file_paths = Path('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/V1_MTG_Data_Copies/mapped_to_v1').glob('*/processed/*.h5ad')


# for file in h5ad_file_paths:
#     if 'updated' not in file.stem:
#         x = read_h5ad(file)
#         h5ad_raws.append(x)
# #combo_anndata.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/Merscope data copies/Trygve MTG V1/Great Apes Data Copies/Great_apes_72722.h5ad')


In [93]:
# ad = read_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/Merscope data copies/Trygve MTG V1/Great Apes Data Copies/Great_apes_72722.h5ad')
# ad.obs[ad.obs['cluster'].str.contains('SEAAD')]


,cluster,subclass,neighborhood,class,merscope,avg.cor,genes_detected,total_reads,prob,volume,section,filename


In [2]:
# old_real = read_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/reference_data/human_merscope_data_michael_2021.h5ad')

In [4]:
# old_real.obs


,cluster_type_label,cell_subclass_label,cluster_label,section,merscope,avg.cor,genes_detected,total_reads,prob,seg_qc
311899265525804364043972895419228225536,glia,Astrocyte,Astro L1-2 FGFR3 GFAP,H19.30.001.Cx46.MTG2.02.007.2.03,VMSC01001,0.322595,27.0,38.0,0.99,Good
69679171148257952179749542001121427456,inh,PVALB,Inh L2-5 PVALB SCUBE3,H19.30.001.Cx46.MTG2.02.007.2.03,VMSC01001,0.168859,23.0,72.0,0.36,Good
239833157268374065721939574161137467392,inh,PVALB,Inh L2-4 PVALB WFDC2,H19.30.001.Cx46.MTG2.02.007.2.03,VMSC01001,0.186431,13.0,39.0,0.40,Good
171122132958009907927092138959567847424,inh,VIP,Inh L1-4 VIP PENK,H19.30.001.Cx46.MTG2.02.007.2.03,VMSC01001,0.198501,18.0,208.0,0.48,Good
129103549955172881216715439196990341120,inh,SST,Inh L3-6 SST NPY,H19.30.001.Cx46.MTG2.02.007.2.03,VMSC01001,0.155167,36.0,177.0,0.43,Good
...,...,...,...,...,...,...,...,...,...,...
136397388990108203821476150230009249792,inh,VIP,Inh L1-4 VIP CHRNA6,H21.33.013.CX24.MTG.02.007.1.06,VMSC01001,0.221619,40.0,59.0,0.58,Good
277079421045952491478657663013379637248,inh,LAMP5/PAX6/Other,Inh L2-6 LAMP5 CA1,H21.33.013.CX24.MTG.02.007.1.06,VMSC01001,0.198087,17.0,62.0,0.67,Good
77817010389189751166486646236481847296,glia,Endothelial,Endo L2-6 NOSTRIN,H21.33.013.CX24.MTG.02.007.1.06,VMSC01001,0.409673,16.0,41.0,0.93,Good
65772286567755543545015010748627156992,inh,VIP,Inh L1-2 VIP TSPAN12,H21.33.013.CX24.MTG.02.007.1.06,VMSC01001,0.178492,39.0,53.0,0.58,Good
